In [24]:
from multiprocessing import Pool, cpu_count
import numpy as np
from src.input.write_files import write_circuit_file
from src.input.read_files import find_files, numbers_from_file_name, normal_sec_size_int_from_file_name, number_from_text
from datetime import datetime
from src.algorithm.circuit_synthesis_vanilla import synthesise_circuit
from src.algorithm.execution import load_circuit_and_benchmark
import csv

print(f'There are {cpu_count()} threads available on this machine.')

There are 12 threads available on this machine.


In [25]:
sample_size = 5
lower_size = 8
upper_size = 100
step_size = 1

sizes = np.arange(lower_size, upper_size + step_size, step_size)
samples = np.arange(1, sample_size + 1, 1)

input_gen_processors = 10

input_dir = r'resources\runtime_performance\input_circuits'
input_file_prefix = 'Circuit'

benchmark_processors = 10

output_dir = r'resources\runtime_performance'
output_file_prefix = 'RuntimePerformance'

output_file_fieldnames = ['size', 'run', 'sec_size', 'num_gates', 'process_time', 'initial_rss', 'final_rss']

In [26]:
input_gen_argument_tuples = ([(size, size * size, rf'..\{input_dir}\{input_file_prefix}_{size}_{sample}.txt') for size in sizes for sample in samples])

[print(argument_tuple) for argument_tuple in input_gen_argument_tuples[:min(10, len(input_gen_argument_tuples))]]

(8, 64, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_1.txt')
(8, 64, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_2.txt')
(8, 64, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_3.txt')
(8, 64, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_4.txt')
(8, 64, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_5.txt')
(9, 81, '..\\resources\\runtime_performance\\input_circuits\\Circuit_9_1.txt')
(9, 81, '..\\resources\\runtime_performance\\input_circuits\\Circuit_9_2.txt')
(9, 81, '..\\resources\\runtime_performance\\input_circuits\\Circuit_9_3.txt')
(9, 81, '..\\resources\\runtime_performance\\input_circuits\\Circuit_9_4.txt')
(9, 81, '..\\resources\\runtime_performance\\input_circuits\\Circuit_9_5.txt')


[None, None, None, None, None, None, None, None, None, None]

In [27]:
print(f'Generating {len(input_gen_argument_tuples)} circuits in size range from {lower_size} to {upper_size} ...')

if __name__ ==  '__main__':
    with Pool(processes = input_gen_processors) as pool:
        pool.starmap(write_circuit_file, input_gen_argument_tuples)

print(f'Saved circuits to {input_dir} ...')

Generating 465 circuits in size range from 8 to 100 ...
Saved circuits to resources\runtime_performance\input_circuits ...


In [28]:
file_regex = rf'({input_file_prefix}).*\.txt'
size_regex = r'_\d+_'
sample_regex = r'_\d+\.'

input_file_names = find_files(rf'..\{input_dir}', file_regex)
input_file_names.sort(key = lambda input_file_name: numbers_from_file_name(input_file_name, size_regex, sample_regex))

[print(input_file_name) for input_file_name in input_file_names[:min(10, len(input_file_names))]]

Circuit_8_1.txt
Circuit_8_2.txt
Circuit_8_3.txt
Circuit_8_4.txt
Circuit_8_5.txt
Circuit_9_1.txt
Circuit_9_2.txt
Circuit_9_3.txt
Circuit_9_4.txt
Circuit_9_5.txt


[None, None, None, None, None, None, None, None, None, None]

In [29]:
file_name_arguments = [(number_from_text(input_file_name, size_regex),
                        normal_sec_size_int_from_file_name(input_file_name, size_regex),
                        number_from_text(input_file_name, sample_regex),
                        input_file_name)
                       for input_file_name in input_file_names]
benchmark_argument_tuples = [(synthesise_circuit,
                              size,
                              sec_size,
                              sample,
                              rf'..\{input_dir}\{input_file_name}')
                             for (size, sec_size, sample, input_file_name ) in file_name_arguments]

[print(argument_tuple) for argument_tuple in benchmark_argument_tuples[:min(10, len(benchmark_argument_tuples))]]

(<function synthesise_circuit at 0x000001F9B9A06A60>, 8, 2, 1, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_1.txt')
(<function synthesise_circuit at 0x000001F9B9A06A60>, 8, 2, 2, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_2.txt')
(<function synthesise_circuit at 0x000001F9B9A06A60>, 8, 2, 3, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_3.txt')
(<function synthesise_circuit at 0x000001F9B9A06A60>, 8, 2, 4, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_4.txt')
(<function synthesise_circuit at 0x000001F9B9A06A60>, 8, 2, 5, '..\\resources\\runtime_performance\\input_circuits\\Circuit_8_5.txt')
(<function synthesise_circuit at 0x000001F9B9A06A60>, 9, 2, 1, '..\\resources\\runtime_performance\\input_circuits\\Circuit_9_1.txt')
(<function synthesise_circuit at 0x000001F9B9A06A60>, 9, 2, 2, '..\\resources\\runtime_performance\\input_circuits\\Circuit_9_2.txt')
(<function synthesise_circuit at 0x000001F9B9A06A60>, 9, 2, 3,

[None, None, None, None, None, None, None, None, None, None]

In [30]:
output_file_timestamp = datetime.now().strftime('%d%m%y_%H%M')

print(f'Benchmarking runtime performance in size range from {lower_size} to {upper_size} in {benchmark_processors} threads ...')

if __name__ ==  '__main__':
    with Pool(processes = benchmark_processors) as pool:
        result_tuples = pool.starmap(load_circuit_and_benchmark, benchmark_argument_tuples)

print(f'Saving results to {output_dir} ...')

with open(rf'..\{output_dir}\{output_file_prefix}_{output_file_timestamp}.csv', mode='w', newline='') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=output_file_fieldnames)
    writer.writeheader()

    for (size, run, sec_size, num_gates, process_time, initial_rss, final_rss) in result_tuples:

        writer.writerow({'size': size, 'run': run, 'sec_size': sec_size, 'num_gates': num_gates, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print('Done.')


Benchmarking runtime performance in size range from 8 to 100 in 10 threads ...
Saving results to 'resources\runtime_performance ...
Done.
